<a href="https://colab.research.google.com/github/kjhyoo/llama2-finetuning/blob/main/chatbot_connect_llama2_finetune_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기 코드 (추가함) 0617 * 0617이후 코드수정기능 제거함 파일-> 드라이브에 사본저장으로 활용해주세요!

In [ ]:
# datasets: loads and manages data from hugging-face's community.
# jsonlines: handles json format data
!pip install datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
# 테스트 데이터 깃허브상의 불러온 데이터임
# 변경포인트: pd.read_csv("파일경로")로 변경 가능함
trainDataPath = "https://raw.githubusercontent.com/hyokwan/llama_repository/main/dataset/indata_kor.csv"

In [ ]:
# Define paths and filenames
# Load Data
try:
    df = pd.read_csv(trainDataPath, encoding="ms949")
except Exception as e:
    df = pd.read_csv(trainDataPath, encoding="utf-8")

In [ ]:
# 빈컬럼 생성 및 컬럼 재정의
df["input"]=""

In [ ]:
df.columns = ["instruction","output","input"]
df.head(2)

,instruction,output,input
0,유튜브 채널 hkcode에서는 무엇을 가르치나요?,"초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.",
1,유튜브 채널 hkcode는 누가 운영하나요?,한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.,


In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# # 필요시 허깅페이스에 업로드
# dataset.push_to_hub("hyokwan/llama3data_hkcode")

# 파인튜닝 코드 일부 수정된 코드 반영 패키지등 업데이트 0617

라이브러리 선언

In [ ]:
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★ 변경포인트 본인의 학습데이터 셋을 hkcode_dataset 변수 내 선언해야함)

In [ ]:
# Hugging Face Basic Model
base_model = "upstage/SOLAR-10.7B-Instruct-v1.0"

# Custom Dataset (본인이 hugging face 내 저장한 모델 경로를 설정!)
hkcode_dataset = "hyokwan/llama3data_hkcode"

new_model = "solar-hkcode-model"

In [ ]:
dataset = load_dataset(hkcode_dataset, split="train")  # 허깅페이스에 있는 데이터셋 불러옴

# 데이터 확인
print( dataset[0] )
print( dataset[4] )
print( dataset[9] )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32 [00:00<?, ? examples/s]

{'instruction': '유튜브 채널 hkcode에서는 무엇을 가르치나요?', 'output': '초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.', 'input': ''}
{'instruction': '스마트금융과는 1년에 몇 명을 선발하나요?', 'output': '1년에 한반을 운영하고 있고 최대 27명을 선발합니다.', 'input': ''}
{'instruction': '스마트금융과는 학과 홈페이지내 트랙에 대한 문의', 'output': '학과 홈페이지내 빅데이터, 인공지능, 블록체인, 웹개발 트랙등이 있는데 모든 트랙에 대해서 기초는 모두 학습합니다. 회사와 매핑이 완료되면 업체향 필요 트랙을 더 추가 교육을 하는 부분입니다', 'input': ''}


3. 데이터 불러오기 (훈련)

In [ ]:
# 솔라 포맷으로 변환하는 함수
### System:\n{input}\n\n### User:\n{instruction}\n### Assistant:\n

def convert_to_solar_format(dataset):
    instruction = dataset['instruction']
    output = dataset['output']

    prompt = f"<s>### User:\n{instruction}\n\n### Assistant:\n{output}</s>"

    return {'prompt': prompt}

In [ ]:
# 데이터셋을 SOLAR 포맷으로 변환
solar_dataset = dataset.map(convert_to_solar_format, remove_columns=['instruction', 'output'])

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
print(solar_dataset[0])

{'input': '', 'prompt': '<s>### User:\n유튜브 채널 hkcode에서는 무엇을 가르치나요?\n\n### Assistant:\n초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.</s>'}


**4**. 4비트 양자화 QLoRA 파인튜닝 (효율성) * 파라미터를 고정 시키고 추가데이터만 튜닝

In [ ]:
# GPU의 Compute Capability가 8 이상이면 최신의 고성능 GPU로 간주
if torch.cuda.get_device_capability()[0] >= 8:  # 현재 사용 중인 CUDA 장치(GPU)의 Compute Capability를 반환. 이 반환 값은 (major, minor) 형식의 튜플로, 여기서 major 값이 8 이상인지를 체크
  !pip install -qqq flash-attn  # Flash Attention은 성능을 극대화하기 위한 고효율 어텐션 구현
  attn_implementation = "flash_attention_2"  # Flash Attention v2를 사용하도록 설정
  torch_dtype = torch.bfloat16  # 데이터 타입으로 bfloat16을 사용. bfloat16은 메모리 사용량을 줄이면서도 성능을 유지할 수 있는 데이터 타입
else:
  attn_implementation = "eahf_YLtfLFonsXPkjgBnlFlieNGJBCYvKFCAqUger"  # Eager 모드의 어텐션 구현을 사용. 이는 더 낮은 성능의 GPU에서도 호환됨
  torch_dtype = torch.float16  # 데이터 타입으로 float16을 사용. 이는 일반적인 반정밀도(float16) 데이터 타입


### QLoRa config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트 정밀도로 로드. 이는 모델의 메모리 사용량을 크게 줄일 수 있음
    bnb_4bit_quant_type="nf4",  # 4비트 양자화 타입을 nf4로 설정. nf4는 특정 양자화 기법을 의미
    bnb_4bit_compute_dtype=torch_dtype,  # 계산에 사용할 데이터 타입을 위에서 선택한 torch.float16 으로 설정. GPU 성능에 따라 bfloat16일 수도 있음
    bnb_4bit_use_double_quant=False,  # 더블 양자화를 사용하지 않음. 더블 양자화는 두 단계로 양자화하는 기법
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.9 MB/s eta 0:00:00


### 5. 모델 불러오기

In [ ]:
model = AutoModelForCausalLM.from_pretrained(  # 사전 학습된 언어 모델을 로드하는 함수! AutoModelForCausalLM은 Causal Language Modeling(자연어 생성)을 위한 사전 학습된 모델을 자동으로 로드하는 클래스
    base_model,  # 사전 학습된 모델의 이름이나 경로 (위에서 선언함! 허깅페이스에 올라온 모델)
    quantization_config=quant_config,  # 위에서 양자화 설정 되어 있는 것 받아옴
    device_map={"": 0}  # 모델을 특정 장치에 할당. 빈 문자열 키 => 전체 모델 / 값 0 => 첫 번째 GPU 장치(또는 CPU)로 모델을 할당함을 의미
    # device_map="auto"  # 모델을 가능한 모든 장치에 자동으로 분산할 수 있음을 나타냄 (여기서는 수동으로 첫 번째 장치에 할당)
)
model.config.use_cache = False  # 모델이 추론 시 캐시를 사용하지 않도록 함. 캐시를 사용하면 모델의 속도가 빨라질 수 있지만, 여기서는 캐시를 사용하지 않도록 설정
model.config.pretraining_tp = 1  # 모델의 pretraining tensor parallelism(사전 학습 시 텐서 병렬화) 설정을 1로 설정. 이는 병렬화를 사용하지 않도록 설정한 것

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [ ]:
# AutoTokenizer : 특정 모델에 해당하는 토크나이저를 자동으로 로드하는 클래스
# AutoTokenizer.from_pretrained : 사전 학습된 토크나이저를 로드하는 함수
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)  # 원격 코드를 신뢰하도록 설정. Hugging Face Hub에 호스팅된 원격 모델 및 토크나이저 코드를 신뢰하고 로드

# tokenizer.eos_token : 문장의 끝을 나타내는 토큰. 패딩 토큰을 문장의 끝 토큰으로 설정하여 문장의 패딩을 수행할 것임을 의미
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 적용할 쪽을 오른쪽으로 설정. 토크나이저가 패딩 토큰을 문장의 끝에 추가할 것임을 나타냄. 즉, 오른쪽에 패딩을 적용
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,  # attention 계산에 사용되는 가중치 행렬의 크기를 나타냄. 일반적으로 큰 값일수록 모델의 표현력이 높아지지만, 계산 비용이 증가
    lora_dropout=0.1,  # attention 계산 시에 적용되는 드롭아웃의 확률을 지정. 드롭아웃은 모델의 과적합을 방지하고 일반화 성능을 향상시키는 데 사용
    r=64,  # attention 계산에 사용되는 쿼리(query), 키(key), 밸류(value)의 차원을 결정. 이 값은 일반적으로 lora_alpha와 관련이 있으며, 주로 모델의 표현력과 계산 비용을 조정하는 데 사용
    bias="none",  # attention 계산의 편향(bias)을 지정. 여기서는 "none"으로 설정되어 있으므로, 어텐션 계산에 편향이 추가되지 않음. 다른 옵션으로는 "query", "key", "value" 등이 있음
    task_type="CAUSAL_LM",  # 모델이 수행할 작업 유형. 여기서는 모델이 인과 언어 모델링(causal language modeling) 작업을 수행. 이는 주어진 이전 토큰들을 기반으로 다음 토큰을 예측하는 작업을 의미
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",  # 학습 결과를 저장할 디렉토리 경로
    num_train_epochs=5,  # 문제지를 몇 번 돌릴 것인지
    per_device_train_batch_size=3,  # 각 학습 장치(device) 당 배치 크기. 더 큰 배치 크기는 학습 속도를 높일 수 있지만, GPU 메모리에 맞게 조정해야 함
    gradient_accumulation_steps=1,  # 그래디언트 누적 단계 수. 배치 크기를 사용하면 GPU 메모리 부족으로 배치를 처리할 수 없을 수 있으므로, 그래디언트 누적을 통해 이를 해결
    optim="paged_adamw_32bit",  # 최적화 알고리즘. paged_adamw_32bit"는 페이지드 AdamW 최적화 알고리즘. 이는 AdamW 최적화 알고리즘의 변형으로, 32비트로 연산을 수행
    save_steps=25,  # 중간 체크포인트 저장 간격. 모델의 중간 상태를 저장하여 학습 중에 중단되었을 때 학습을 재개
    logging_steps=25,  # 학습 로그 출력 간격. 학습 중간에 로그를 출력하여 모델의 학습 진행 상황을 모니터링
    learning_rate=2e-4,  # 초기 학습률. 학습률은 모델이 가중치를 업데이트하는 속도를 결정
    weight_decay=0.001,  # 가중치 감쇠 정도. 중치 감쇠는 모델의 복잡도를 줄이고 과적합을 방지하는 데 도움
    fp16=False,  # mixed precision 학습 사용 여부 (16비트/32비트). True로 설정하면 16비트 부동소수점 형식을 사용하여 학습 속도를 높임
    bf16=False,  # bfloat16 precision 학습 사용 여부 (16비트/32비트)
    max_grad_norm=0.3,  # 그래디언트 클리핑을 위한 최대 그래디언트 노름. 그래디언트 클리핑은 그래디언트 폭주를 방지하고 안정적인 학습을 위해 사용
    max_steps=-1,  # 최대 학습 스텝 수 (-1이면 무제한). 학습 스텝 수가 지정된 값에 도달하면 학습이 중단
    warmup_ratio=0.03,  # 학습률 스케줄러의 초기 웜업 비율. 웜업 단계에서는 초기 학습률을 점진적으로 증가시켜 안정적인 학습을 유도
    group_by_length=True,  # 그룹 당 장치 개수. 분산 학습을 위해 사용
    lr_scheduler_type="constant",  # 학습률 스케줄러 타입. 여기서는 "constant"로 설정되어 있으며, 학습률을 일정한 상수 값으로 유지
    # per_device_group_size=True,  # 그룹 당 장치 개수. 분산 학습을 위해 사용
    # learning_rate_scheduler_type="constant",  # 학습률 스케줄러 타입. 여기서는 "constant"로 설정되어 있으며, 학습률을 일정한 상수 값으로 유지
    report_to="tensorboard"  # 학습 결과 보고서를 전송할 위치. TensorBoard를 사용하여 학습 과정을 모니터링
)

### 8. 모델 훈련

In [ ]:
trainer = SFTTrainer(
    model=model,  # 학습할 모델 (사전 학습된 모델)
    train_dataset=solar_dataset,  # 학습 데이터셋 (새로 학습할 추가 데이터셋)
    peft_config=peft_params,  # SFT 모델의 설정
    dataset_text_field="prompt",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=None,  # 최대 시퀀스 길이
    tokenizer=tokenizer,  # 토크나이저
    args=training_params,  # 학습 인자들 (파라미터 설정 넣어주는 부분?)
    packing=False,  # 패킹 사용 여부
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,1.902100
50,0.632000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=55, training_loss=1.1930389360948042, metrics={'train_runtime': 166.0929, 'train_samples_per_second': 0.963, 'train_steps_per_second': 0.331, 'total_flos': 1347322170654720.0, 'train_loss': 1.1930389360948042, 'epoch': 5.0})

### 검증

In [ ]:
# https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0

In [ ]:
conversation = [ {'role': 'user', 'content': '스마트금융과는 어디에 위치하나요?'} ]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
스마트금융과는 어디에 위치하나요?

### Assistant:
서울시 강서구쪽에 위치하며 지하철 5호선 우장산역에서 걸어서 20분 거리에 위치하고 있습니다.</s>


In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)

def extract_response_solar(input, prompt):
    result = pipe(f"### System:\n{input}\n\n### User:\n{prompt}\n\n### Assistant:\n",
                do_sample = True,  # 샘플링 방법을 지정. True : 모델이 확률에 따라 텍스트를 샘플링
                top_p = 0.9,  # 샘플링할 때 사용되는 확률 분포의 상위 일부를 지정
                temperature = 0.1,  # 다양성을 조절
                num_return_sequences = 1,  # 생성할 텍스트 시퀀스의 수를 지정
                eos_token_id = tokenizer.eos_token_id,  # 문장의 끝을 나타내는 토큰의 ID를 지정
                max_length = 4096)  # 생성된 텍스트의 최대 길이

    print(result[0]['generated_text'].split("Assistant:\n")[-1])

In [ ]:
!python --version

Python 3.10.12


In [ ]:
# input = "단답형의 단어로 대답해"
prompt = "HKCODE 유튜브 채널은 누가 운영하나요?"

extract_response_solar(input, prompt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.


### 모델 저장 (구글드라이브 연동 및 타겟 위치 지정)

In [ ]:
from google.colab import drive
import os

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
basePath = "/content/gdrive/MyDrive/Colab Notebooks/llama"
newModelPath = os.path.join(basePath,new_model)

In [ ]:
# 어댑터 저장 (향후 재사용 시 베이스모델 + 어댑터 저장해서 활용해야함)
trainer.save_model(newModelPath)  # 위에서 선언한 모델명으로 로컬 저장

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import gc

# # Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()